Required pip installs

In [ ]:
!pip install -U aeon
!pip install keras-self-attention

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.0.3 which is incompatible.
mizani 0.13.0 requires pandas>=2.2.0, but you have pandas 2.0.3 which is incompatible.
plotnine 0.14.1 requires pan

# COSCO: A Sharpness-Aware Training Framework for Few-shot Multivariate Time Series Classification

This notebook is a preview of the COSCO few-shot method. Please replace datasets with desired classification tasks. Current COSCO logic is used with ResNet, but can be applied to any Deep Learning model.


## Parameters
- LR: Learning Rate of model, default 0.01
- rho: Neighborhood parameter for SAM, default 0.1
- nEpoch: Number of epochs, default 100

Chnage parameters you would like to explore

In [ ]:
import argparse
def get_args():
    parser = argparse.ArgumentParser()
    #Hyperparameters
    parser.add_argument('--lr', type=float, default=0.01)
    parser.add_argument('--rho', type=float, default=0.1)
    parser.add_argument('--nEpoch', type=int, default=100)

    #Data Loading
    parser.add_argument('--dataset', type=str, default='BasicMotions')
    parser.add_argument('--shot', type=int, default=1 ,choices=[1,10])
    parser.add_argument('--normalize', type=bool, default=False)

    #Baseline Model
    parser.add_argument('--model', type=str, default='resnet',choices=['resnet','tapnet'])

    #SAM
    parser.add_argument('--sam', type=bool, default=True)
    parser.add_argument('--optimizer', type=str, default='adam',choices=['sgd','adam'])

    #Prototypical Loss
    parser.add_argument('--prototypical_loss', type=bool, default=True)


    #Other Parameters
    parser.add_argument('--prototypical_loss_type',type=str, default='neg',choices=['neg','sim','cos','negexp'])

    #Saving
    parser.add_argument('--save_dir', type=str, default='/content/classification_data/')
    parser.add_argument('--save_name', type=str, default='results.csv')

    return parser.parse_args(args=[])


args = get_args()
print(args)

Namespace(lr=0.01, rho=0.1, nEpoch=100, dataset='BasicMotions', shot=1, normalize=False, model='resnet', sam=True, optimizer='adam', prototypical_loss=True, prototypical_loss_type='neg', save_dir='/content/classification_data/', save_name='results.csv')


Imports

In [ ]:
!git clone https://github.com/JRB9/COSCO.git

Cloning into 'COSCO'...
remote: Enumerating objects: 445, done.
remote: Counting objects: 100% (291/291), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 445 (delta 75), reused 232 (delta 41), pack-reused 154 (from 1)
Receiving objects: 100% (445/445), 82.32 MiB | 9.31 MiB/s, done.
Resolving deltas: 100% (108/108), done.
Updating files: 100% (225/225), done.
Filtering content: 100% (80/80), 2.06 GiB | 58.85 MiB/s, done.
Encountered 130 file(s) that should have been pointers, but weren't:
	Datasets/ArticularyWordRecognition/1-shot/X_train.npy
	Datasets/ArticularyWordRecognition/1-shot/original_indices.npy
	Datasets/ArticularyWordRecognition/1-shot/y_train.npy
	Datasets/ArticularyWordRecognition/10-shot/X_train.npy
	Datasets/ArticularyWordRecognition/10-shot/original_indices.npy
	Datasets/ArticularyWordRecognition/10-shot/y_train.npy
	Datasets/BasicMotions/1-shot/X_train.npy
	Datasets/BasicMotions/1-shot/original_indices.npy
	Datasets/BasicMotions/1-shot/y_train.np

In [ ]:
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.nn.modules.batchnorm import _BatchNorm
import pandas as pd
import torch.nn as nn
import os
import uuid
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

Prototypical Loss

In [ ]:
from COSCO.Prototypical_Loss import PrototypicalLoss

In [ ]:
from COSCO.Prototypical_Loss import prototypical_testing as ptest

ResNet

In [ ]:
from COSCO.Baselines.ResNet import *

Tapnet

In [ ]:
from COSCO.Baselines.TapNet import *

/usr/local/lib/python3.10/dist-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


SAM Optimizer

In [ ]:
from COSCO.SAM import SAM

Loading and Savind Data

In [ ]:
from COSCO.utils.load_data import *
from COSCO.utils.save import *

Importing the training model

In [ ]:
from COSCO.utils.proto_model import *

Start training

In [ ]:
import torch
import torch.nn as nn
# Instead of importing _BatchNorm, import the appropriate BatchNorm class
from torch.nn import BatchNorm1d, BatchNorm2d, BatchNorm3d

def disable_running_stats(model):
    def _disable(module):
        # Check if the module is an instance of any of the BatchNorm classes
        if isinstance(module, (BatchNorm1d, BatchNorm2d, BatchNorm3d)):
            module.backup_momentum = module.momentum
            module.momentum = 0

    model.apply(_disable)

def enable_running_stats(model):
    def _enable(module):
        # Check if the module is an instance of any of the BatchNorm classes
        if isinstance(module, (BatchNorm1d, BatchNorm2d, BatchNorm3d)) and hasattr(module, "backup_momentum"):
            module.momentum = module.backup_momentum

    model.apply(_enable)

In [ ]:
#Same thing you have to make these all files

def full_training(args):
  train_data, train_label, test_data, test_label = load_data(args)

  traindata = Dataset(train_data ,train_label)

  input_size = train_data.shape[-1]

  if args.model == "tapnet":
     test_label =test_label.reshape(-1)
     train_label = train_label.reshape(-1)

  elif args.model=="resnet":
    batch_size = 1024
    trainloader = DataLoader(traindata, batch_size=batch_size,
                         shuffle=True, num_workers=2)
  acc = []
  for i in range(5):
    if args.model=='tapnet':
        acc_tmp = train_tapnet(train_data, train_label, test_data, test_label, input_size,args)
    elif args.model =='resnet':
       acc_tmp = proto_neg_train_model(trainloader, train_label, test_data, test_label, input_size,args)
    print(i)
    acc.append(acc_tmp)

  acc = np.array(acc)

  # save the data
  save_to_file_directory(acc,args)

  # save to dataframe
  save_to_dataframe(acc,args)

  return acc

In [ ]:
# columns for our results dataframe
columns = ["Dataset", "Shots", "Normalization", "Result"]

# dataframe construction
df = pd.DataFrame(columns = columns)

# filepath for our csv
filepath = args.save_dir + args.save_name

os.makedirs(os.path.dirname(filepath), exist_ok=True)

# creating empty df and csv
df.to_csv(filepath, index=False)

for dataset_name in [args.dataset]:
  for shot_dir in [args.shot]:
    full_training(args)


/content/COSCO/utils/proto_model.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs1 = model_resnet(torch.tensor(inputs).transpose(1,2))


Epoch: 1 --> 0.7946372032165527 0.7946372032165527 0.7926415801048279


/content/COSCO/utils/proto_model.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tmp = criterion(model_resnet(torch.tensor(inputs).transpose(1,2).float())[1], labels)


Epoch: 2 --> 0.7940773963928223 0.7940773963928223 0.8071625232696533
Epoch: 3 --> 0.786394476890564 0.786394476890564 0.7703090906143188
Epoch: 4 --> 0.7779690027236938 0.7779690027236938 0.7599457502365112
Epoch: 5 --> 0.7714455127716064 0.7714455127716064 0.7559036016464233
Epoch: 6 --> 0.7668169736862183 0.7668169736862183 0.7566694617271423
Epoch: 7 --> 0.7624586224555969 0.7624586224555969 0.760635495185852
Epoch: 8 --> 0.7588100433349609 0.7588100433349609 0.7561649084091187
Epoch: 9 --> 0.7561074495315552 0.7561074495315552 0.7547019124031067
Epoch: 10 --> 0.7541137933731079 0.7541137933731079 0.7533724904060364
Epoch: 11 --> 0.7526184320449829 0.7526184320449829 0.752106249332428
Epoch: 12 --> 0.7514823079109192 0.7514823079109192 0.7512405514717102
Epoch: 13 --> 0.7505436539649963 0.7505436539649963 0.7508639097213745
Epoch: 14 --> 0.7497263550758362 0.7497263550758362 0.7520103454589844
Epoch: 15 --> 0.7489176988601685 0.7489176988601685 0.751502275466919
Epoch: 16 --> 0.748

/content/COSCO/utils/proto_model.py:137: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_centroids = torch.load('train_centroids.pt')


Epoch: 1 --> 0.7814762592315674 0.7814762592315674 0.7740033864974976
Epoch: 2 --> 0.7838499546051025 0.7838499546051025 0.769862174987793
Epoch: 3 --> 0.784179151058197 0.784179151058197 0.7691371440887451
Epoch: 4 --> 0.7743150591850281 0.7743150591850281 0.758507490158081
Epoch: 5 --> 0.7624194622039795 0.7624194622039795 0.7542306780815125
Epoch: 6 --> 0.7558296918869019 0.7558296918869019 0.7534761428833008
Epoch: 7 --> 0.7526492476463318 0.7526492476463318 0.7510681748390198
Epoch: 8 --> 0.7510296702384949 0.7510296702384949 0.7494230270385742
Epoch: 9 --> 0.7501491904258728 0.7501491904258728 0.7482689619064331
Epoch: 10 --> 0.7496393918991089 0.7496393918991089 0.7475621700286865
Epoch: 11 --> 0.7492998838424683 0.7492998838424683 0.7470522522926331
Epoch: 12 --> 0.7490816712379456 0.7490816712379456 0.7465749979019165
Epoch: 13 --> 0.7489213943481445 0.7489213943481445 0.7461759448051453
Epoch: 14 --> 0.7487688064575195 0.7487688064575195 0.7459506988525391
Epoch: 15 --> 0.748